<picture> <img align="center" src="https://hermes.digitalinnovation.one/assets/diome/logo-full.svg" width = 150px></picture>

**Autor:** Israel de Castro Evangelista

**Profissão:** Data Science and Analyst

**Bootcamp:** Santander Bootcamp 2023 - Ciência de Dados com Python

**Projeto:** Explorando IA Generativa em um Pipeline de ETL com Python

**Introdução**

Este projeto está focado em aprimorar habilidades relacionadas a prompts de comando para IA Generativa, como o chat GPT, visando alcançar resultados mais precisos, alinhados com os objetivos propostos. Um dos principais pilares deste projeto é a aplicação de técnicas de ETL (Extração, Transformação e Carregamento). Em ciência de dados, o ETL é vital para coletar informações de uma ou diversas fontes, realizar uma limpeza criteriosa, transformar os dados conforme a necessidade e, finalmente, carregar esses dados para sua destinação final, seja ela uma base consolidada, data warehouse ou qualquer outra plataforma de armazenamento.

**Metodologia do Projeto**

Diferente do proposto no projeto incial, foi usado o chat GPT-4, juntamente com o Code Interpreter, para gerar uma base de dados que simula a de um empreendimento bancário. Foram criados prompts iterativos para performar o resultado desejado, como detalhado na seção de "Extração dos dados". O Code Interpreter auxiliou na geração da planilha Excel, que posteriormente foi hospedada no GitHub.

A partir deste ponto, começa o processo de ETL propriamente dito.

- **Extração:** Base coletada a partir do repositório no Github.
- **Transformação:** Algumas etapas de tratamento foram executadas. Para a geração de mensagens utilizando a IA Generativa, optei por não usá-la, pois já havia usado em outros projetos a API da openai. Uma nova solução foi proposta para o mesmo resultado.
- **Carregamento:** Transformação das alterações em excel para que fosse persistido na base original no github.

# Extração dos dados

**Prompt de criação de Tabelas Base no GPT - Code Interpreter**

Para criar as tabelas no contexto de um banco, precisamos de sua autorização. Você autoriza a criação destas tabelas conforme descrito?

Se sim, prosseguiremos da seguinte forma:

1. Criaremos a tabela de cadastro de usuários com os seguintes atributos:
   - ID
   - Nome
   - Idade
   - Sexo
   - CPF (formatado como ***.***.***-XX)
   
2. Criaremos a tabela de dados do cartão com os seguintes atributos:
   - ID (referente ao usuário)
   - Número do cartão (com mascaramento **** **** **** XXXX)
   - Bandeira

   Nota: Alguns usuários podem ter mais de um cartão associado.

3. Criaremos a tabela de mensagens enviadas com os seguintes atributos:
   - ID (inicialmente nulo, referente ao usuário)
   - Data de envio do anúncio (inicialmente vazia)
   - Anúncio (inicialmente vazio)

Após a criação, exibiremos as tabelas para sua validação. Caso esteja tudo correto, salvaremos em um arquivo Excel, onde cada tabela será uma aba distinta. Caso contrário, ajustaremos conforme suas instruções.

Você autoriza a prosseguir?

In [1]:
# Com a execução do prompt acima e uso do Code Interpreter, a base em excel foi gerada e carregada para o Github

In [1]:
import pandas as pd

# Base de dados gerado pelo prompt GPT-4, importadas de arquivo excel via Github
url = "https://raw.githubusercontent.com/IsraelEvangelista/Santander_Bootcamp_2023/main/Projeto%20ETL%20com%20Python%20com%20IA%20Generativa/BD_ETL_Santander2023.xlsx"
df1 = pd.read_excel(url, sheet_name='Cadastro de Usuários')
df2 = pd.read_excel(url, sheet_name='Dados do Cartão')
df3 = pd.read_excel(url, sheet_name='Mensagens Enviadas')

# Transformação dos Dados

In [2]:
# Analisando formato de cada atributo das tabelas

print(f"Cadastro de Usuários")
print(df1.dtypes)
print("\n")
print(f"Dados do Cartão")
print(df2.dtypes)
print("\n")
print(f"Mensagens Enviadas")
print(df3.dtypes)

Cadastro de Usuários
ID        int64
Nome     object
Idade     int64
Sexo     object
CPF      object
dtype: object


Dados do Cartão
ID                   int64
Número do cartão    object
Bandeira            object
dtype: object


Mensagens Enviadas
ID                          object
Data de envio do anúncio    object
Anúncio                     object
dtype: object


In [3]:
# Para que possamos integrar a informação de todas as tabelas, é necessário que o mesmo atributo possua o mesmo formato em todas elas.
# Neste caso, vamos transformar 'ID' da Tabela df3 de obj para int

df3['ID'] = df3['ID'].astype('int64')
df3["Data de envio do anúncio"] = pd.to_datetime(df3["Data de envio do anúncio"])

In [4]:
df3.dtypes

ID                                   int64
Data de envio do anúncio    datetime64[ns]
Anúncio                             object
dtype: object

In [5]:
# Agora vamos dar um Merge nas três tabelas utilizando o atributo da chave primária 'ID'

df = pd.merge(df1, df2, on="ID", how="outer")
df = pd.merge(df, df3, on="ID", how="outer")
df

,ID,Nome,Idade,Sexo,CPF,Número do cartão,Bandeira,Data de envio do anúncio,Anúncio
0,1,Ana Silva,25,Feminino,***.***.***-37,**** **** **** 3307,Visa,NaT,NaN
1,1,Ana Silva,25,Feminino,***.***.***-37,**** **** **** 5983,Visa,NaT,NaN
2,2,João Costa,30,Masculino,***.***.***-85,**** **** **** 3151,Elo,NaT,NaN
3,3,Maria Santos,27,Feminino,***.***.***-34,**** **** **** 5209,MasterCard,NaT,NaN
4,3,Maria Santos,27,Feminino,***.***.***-34,**** **** **** 2733,Visa,NaT,NaN
5,4,Lucas Ferreira,29,Masculino,***.***.***-75,**** **** **** 5916,Amex,NaT,NaN
6,5,Fernanda Oliveira,31,Feminino,***.***.***-14,**** **** **** 5642,Elo,NaT,NaN


Gerando múltiplas saídas de anúncios para envio aos clientes

In [6]:
import random
import datetime

# Ampliando a lista de frases-chave
frases_base = [
    "{Nome}, conquiste seus sonhos",
    "{Nome}, invista no futuro",
    "{Nome}, dê o primeiro passo",
    "{Nome}, sua chance de brilhar",
    "Oportunidade única para {Nome},",
    "{Nome}, transforme suas finanças",
    "Seu futuro começa agora, {Nome},",
    "{Nome}, faça o dinheiro trabalhar para você",
    "A jornada rumo à prosperidade espera por {Nome},",
    "{Nome}, realize mais, invista conosco"
]

# Ampliando a lista de chamadas à ação (CTA)
ctas = [
    " com nosso título de capitalização!",
    "! Adquira já e sinta a diferença!",
    " com as melhores taxas do mercado!",
    ". Venha conferir nossos planos!",
    "! Não perca esta chance!",
    ". Aproveite enquanto é tempo!",
    "! Junte-se aos vencedores!",
    ". Uma decisão inteligente!",
    "! Seu futuro financeiro aguarda!",
    ". Estabilidade e confiança aqui!"
]

# Função adaptada para gerar anúncio personalizado usando um registro de usuário
def gerar_anuncio_personalizado(user):
    frase = random.choice(frases_base).format(Nome=user['Nome']) + random.choice(ctas)
    while len(frase) > 150:  # Garantir que o anúncio tenha no máximo 150 caracteres
        frase = random.choice(frases_base).format(Nome=user['Nome']) + random.choice(ctas)
    return frase

# Iterando sobre 10 registros aleatórios de df1 e gerando anúncios personalizados
amostra = df1.sample(n=min(10, len(df1)))

for _, user in amostra.iterrows():
    anuncio = gerar_anuncio_personalizado(user)
    data_envio = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    novo_registro = {"ID": user["ID"], "Data de envio do anúncio": data_envio, "Anúncio": anuncio}
    df3 = df3.append(novo_registro, ignore_index=True)

# Mostrando os novos registros em df3
df3.tail(len(amostra))


<ipython-input-6-8dac6f8fbb9f>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(novo_registro, ignore_index=True)
<ipython-input-6-8dac6f8fbb9f>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(novo_registro, ignore_index=True)
<ipython-input-6-8dac6f8fbb9f>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(novo_registro, ignore_index=True)
<ipython-input-6-8dac6f8fbb9f>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(novo_registro, ignore_index=True)
<ipython-input-6-8dac6f8fbb9f>:46: FutureWarning: The frame.append method is deprecated and will be removed from pan

,ID,Data de envio do anúncio,Anúncio
0,1,2023-08-18 13:53:39,"Ana Silva, invista no futuro com nosso título ..."
1,4,2023-08-18 13:53:39,"Oportunidade única para Lucas Ferreira,! Adqui..."
2,5,2023-08-18 13:53:39,"Fernanda Oliveira, faça o dinheiro trabalhar p..."
3,3,2023-08-18 13:53:39,"Maria Santos, realize mais, invista conosco! A..."
4,2,2023-08-18 13:53:39,"João Costa, sua chance de brilhar! Não perca e..."


# Carregando os Dados

In [8]:
with pd.ExcelWriter('BD_ETL_Santander2023_modificado.xlsx', engine='openpyxl') as writer:
    df1.to_excel(writer, sheet_name='Cadastro de Usuários', index=False)
    df2.to_excel(writer, sheet_name='Dados do Cartão', index=False)
    df3.to_excel(writer, sheet_name='Mensagens Enviadas', index=False)


In [9]:
from google.colab import files
files.download('BD_ETL_Santander2023_modificado.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>